# Example: XRD spectra

### Importing necessary packages

In [ ]:
from datetime import datetime
from sdRDM import DataModel
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import re
import pickle

### Loading data model from GitHub

In [ ]:
url_example_showcase = "https://github.com/FAIRChemistry/example_showcase.git"

In [ ]:
model = DataModel.from_git(url=url_example_showcase)

### Parsing XRD data

In [ ]:
content = {}
for line in open('XRD_data_set.txt', 'r'):
    line = line.strip()
    if '=' in line:
        key_value = re.split('=', line.strip(r'_'))
        try:
            content[key_value[0]] = float(key_value[1].strip("'"))
        except ValueError:
            content[key_value[0]] = key_value[1].strip("'")
for key, value in content.items():
    print(key, ':', value)


In [ ]:
from io import StringIO


raw_data = ''
for line in open('XRD_data_set.txt', 'r'): 
    if '=' not in line and ';' not in line and '_' not in line:
        raw_data= raw_data + line
#print(raw_data)
data = pd.read_csv(StringIO(raw_data), sep='      ', names=['Angle', 'Intensity'])
print(data)

### Working with the data model

Extracting date and time.


In [ ]:
datemeasured = content['DATEMEASURED']
pattern_date = r"([0-9]{1,2})\-([A-Za-z]*)\-([0-9]{4})\s([0-9]{2})\:([0-9]{2})\:([0-9]{2})"
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
date_raw = re.findall(pattern_date, datemeasured)[0]
datum = []
for part in date_raw:
    try:
        datum.append(int(part))
    except ValueError:
        datum.append(months.index(part.lower())+1)
print(datum)
date = datetime(datum[2], datum[1], datum[0], datum[3], datum[4], datum[5])


Extracting more metadata.

In [ ]:
pattern_wl = r"(WL[1-9]{1})"
wls = [(content[key]) for key in [item.groups()[0] for item in [ re.search(pattern_wl, key) for key in content if re.search(pattern_wl, key) is not None ]]]
wl_ratio = content['WLRATIO']

In [ ]:
sample_id = content['SAMPLE']

In [ ]:
anode = content['ANODE']

In [ ]:
steptime = content['STEPTIME']
stepsize = float(content['STEPSIZE'])
start = float(content['START'])

In [ ]:
theta = content['THETA']
theta2 = float(content['2THETA'])

Feeding the data model with information.

In [ ]:
SpecInf = model.SpecInf(wl = wls, wl_ratio = wl_ratio, anode = anode, steptime = steptime, stepsize = stepsize)

In [ ]:
Dataset = model.Dataset(name_exp = 'Richard Schoemig', date = date, compound = 'Merlinoite', sample_id = sample_id, spec_inf = SpecInf)

In [ ]:
print(Dataset.yaml())

Plotting

In [ ]:

sns.set_theme(rc={'axes.facecolor':'darkslategray', 'figure.facecolor':'goldenrod', 'grid.color':'black', 'axes.edgecolor':'black'})
sns.set_context("notebook", font_scale=1.5)
sns.lineplot(x = "Angle", y = "Intensity", data=data, color = 'crimson')
